In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import model_selection
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ml/Hitters.csv")
df.dropna(inplace=True)

In [6]:
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [7]:
!pip install catboost
from catboost import CatBoostRegressor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.5 MB/s eta 0:00:00


In [13]:
catb_model = CatBoostRegressor(verbose=0).fit(X_train, y_train)

In [14]:
y_pred = catb_model.predict(X_test)

In [15]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

351.194631344607

In [16]:
catb_model.get_params()

{'loss_function': 'RMSE', 'verbose': 0}

**Model Tuning**

In [17]:
catb_params = {
    'iterations': [200,500],
    'learning_rate': [0.01,0.05],
    'depth': [3,5]}

In [18]:
catb_cv_model = GridSearchCV(catb_model,
                             catb_params,
                             cv=5, n_jobs=-1,
                             verbose=2).fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [19]:
catb_cv_model.best_params_

{'depth': 3, 'iterations': 500, 'learning_rate': 0.05}

**Tuned Model**

In [21]:
catb_tuned = CatBoostRegressor(iterations = 500,
                             learning_rate = 0.05,
                             depth = 3,
                             verbose=0).fit(X_train, y_train)

In [22]:
y_pred = catb_tuned.predict(X_test)

In [23]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

328.8622075674009